In [2]:
import numpy as np
from scipy.optimize import minimize

# t: times (to default or censor)
# e: events (1=default, 0=censored)

def fit_exponential(t, e):
    t = np.asarray(t, float); e = np.asarray(e, int)
    d = e.sum(); T = t.sum()
    lam = d / T  # MLE
    # SE and 95% CI (Wald, log-scale for positivity)
    se = lam / np.sqrt(d) if d > 0 else np.nan
    lo = lam * np.exp(-1.96/np.sqrt(d)) if d > 0 else np.nan
    hi = lam * np.exp(+1.96/np.sqrt(d)) if d > 0 else np.nan
    return {"lambda": lam, "se": se, "ci95": (lo, hi)}

# Weibull parameterization: S(t)=exp(-(λ t)^k). Optimize θ=(log k, log λ).
def _weibull_nll(theta, t, e):
    lk, ll = theta
    k, lam = np.exp(lk), np.exp(ll)
    eps = 1e-12
    logt = np.log(np.clip(t, eps, None))
    term = (lam * t) ** k
    # events: log f = log k + k log λ + (k-1)log t - (λ t)^k
    # censored: log S = -(λ t)^k
    ll_events = e * (lk + k*ll + (k-1)*logt - term)
    ll_cens = (1 - e) * (-term)
    return -(ll_events + ll_cens).sum()

def fit_weibull(t, e):
    t = np.asarray(t, float); e = np.asarray(e, int)
    # init from exponential (k≈1, λ≈d/Σt)
    lam0 = max(e.sum()/max(t.sum(), 1e-9), 1e-6)
    x0 = np.array([0.0, np.log(lam0)])
    opt = minimize(_weibull_nll, x0, args=(t, e), method="BFGS")
    lk, ll = opt.x
    k, lam = float(np.exp(lk)), float(np.exp(ll))
    # SEs and 95% CIs from inverse Hessian on log-scale (delta method)
    Hinv = opt.hess_inv if hasattr(opt, "hess_inv") else None
    if Hinv is None or not np.all(np.isfinite(Hinv)):
        return {"k": k, "lambda": lam, "se": (np.nan, np.nan), "ci95": ((np.nan, np.nan),(np.nan, np.nan))}
    se_logk = np.sqrt(Hinv[0,0]) if Hinv[0,0] > 0 else np.nan
    se_logl = np.sqrt(Hinv[1,1]) if Hinv[1,1] > 0 else np.nan
    k_ci = (k * np.exp(-1.96*se_logk), k * np.exp(+1.96*se_logk)) if np.isfinite(se_logk) else (np.nan, np.nan)
    l_ci = (lam * np.exp(-1.96*se_logl), lam * np.exp(+1.96*se_logl)) if np.isfinite(se_logl) else (np.nan, np.nan)
    se_k = k * se_logk if np.isfinite(se_logk) else np.nan
    se_l = lam * se_logl if np.isfinite(se_logl) else np.nan
    return {"k": k, "lambda": lam, "se": (se_k, se_l), "ci95": (k_ci, l_ci)}

# Predict cumulative default probability by time t*:
def pd_exponential(t_star, lam): return 1 - np.exp(-lam * t_star)
def pd_weibull(t_star, k, lam):  return 1 - np.exp(-(lam * t_star)**k)

# ---- minimal demo ----
if __name__ == "__main__":
    # toy data (some defaults, some censored)
    t = np.array([3, 5, 7, 4, 10, 12, 6, 9, 8, 11], float)
    e = np.array([1, 1, 0, 1, 0, 0, 1, 0, 1, 0], int)

    exp_fit = fit_exponential(t, e)
    wei_fit = fit_weibull(t, e)

    print("Exponential:", exp_fit)
    print("Weibull:", wei_fit)
    print("PD(12m) exp:", pd_exponential(12, exp_fit["lambda"]))
    print("PD(12m) wei:", pd_weibull(12, wei_fit["k"], wei_fit["lambda"]))

Exponential: {'lambda': 0.06666666666666667, 'se': 0.029814239699997195, 'ci95': (0.027748073953428685, 0.16017127718139396)}
Weibull: {'k': 1.7141118249302998, 'lambda': 0.08459780643497526, 'se': (0.6797771773689878, 0.024124091707428096), 'ci95': ((0.7878908682075544, 3.7291704561194092), (0.048375357375136326, 0.14794286268752085))}
PD(12m) exp: 0.5506710358827784
PD(12m) wei: 0.6416159462379165
